In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from sklearn.model_selection import train_test_split

# Define paths
data_dir = 'Images'  # Parent folder containing all character folders

# Create image generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    validation_split=0.2
)

# Create generators
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Build CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20
)

# Save the model
model.save('devanagari_character_model.h5')


Found 73600 images belonging to 46 classes.
Found 18400 images belonging to 46 classes.
Epoch 1/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2300/2300 ━━━━━━━━━━━━━━━━━━━━ 271s 118ms/step - accuracy: 0.4303 - loss: 2.0943 - val_accuracy: 0.7853 - val_loss: 0.7196
Epoch 2/20
2300/2300 ━━━━━━━━━━━━━━━━━━━━ 0s 18us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/20
   3/2300 ━━━━━━━━━━━━━━━━━━━━ 1:31 40ms/step - accuracy: 0.8108 - loss: 0.6667

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


2300/2300 ━━━━━━━━━━━━━━━━━━━━ 2332s 1s/step - accuracy: 0.8072 - loss: 0.6213 - val_accuracy: 0.8615 - val_loss: 0.4629
Epoch 4/20
2300/2300 ━━━━━━━━━━━━━━━━━━━━ 0s 4us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/20
2300/2300 ━━━━━━━━━━━━━━━━━━━━ 469s 204ms/step - accuracy: 0.8647 - loss: 0.4387 - val_accuracy: 0.8799 - val_loss: 0.4080
Epoch 6/20
2300/2300 ━━━━━━━━━━━━━━━━━━━━ 0s 4us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/20
2300/2300 ━━━━━━━━━━━━━━━━━━━━ 2637s 1s/step - accuracy: 0.8938 - loss: 0.3458 - val_accuracy: 0.8954 - val_loss: 0.3567
Epoch 8/20
2300/2300 ━━━━━━━━━━━━━━━━━━━━ 0s 4us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/20
2300/2300 ━━━━━━━━━━━━━━━━━━━━ 3677s 2s/step - accuracy: 0.9057 - loss: 0.3086 - val_accuracy: 0.8974 - val_loss: 0.3528
Epoch 10/20
2300/2300 ━━━━━━━━━━━━━━━━━━━━ 0s 7us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/20
2300/2300 ━━━━━━━━━━━━━━━━━━━━ 1734s 754ms/step - accuracy: 0.9201 - loss: 0.2667 - 

In [2]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc:.4f}')

575/575 ━━━━━━━━━━━━━━━━━━━━ 24s 41ms/step - accuracy: 0.9179 - loss: 0.2758
Test accuracy: 0.9154


In [3]:
# Function to predict a single image
def predict_character(image_path, model):
    img = tf.keras.preprocessing.image.load_img(
        image_path, target_size=(64, 64)
    )
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, 0) / 255.0
    
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    
    # Map class index to character name
    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    return class_indices[predicted_class]

# Example usage
image_path = 'Images/character_09_jha/4439.png'
character = predict_character(image_path, model)
print(f'Predicted character: {character}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
Predicted character: character_09_jha
